In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models , initializers
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.optimizers import legacy
import pandas as pd
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import shutil
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


2024-03-19 15:25:46.265280: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-19 15:25:46.724809: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 15:25:46.724897: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 15:25:46.799496: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 15:25:46.950300: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def load_images_from_dir(directory, target_size=(320, 320)):
    image_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(directory, filename)
            # Load the image using OpenCV
            image = cv2.imread(image_path)
            if image is None:
                print(f"Warning: Unable to load image '{filename}'")
                continue
            
            # Convert BGR to RGB format
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Resize the image
            resized_image = cv2.resize(image_rgb, target_size)
            image_list.append(resized_image)
    return image_list



In [3]:
images = load_images_from_dir('for_FID/images')
real_imgs = images[:300]
fake_imgs = images[-300:]

In [4]:
real_imgs_tensor = torch.tensor(real_imgs)
fake_imgs_tensor = torch.tensor(fake_imgs)
print(real_imgs_tensor.shape)

torch.Size([300, 320, 320, 3])


/tmp/ipykernel_145800/1508531010.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  real_imgs_tensor = torch.tensor(real_imgs)


In [5]:
import torch
from torchvision.models import inception_v3
from torchvision import transforms
import torch.nn.functional as F
import numpy as np

def preprocess_image(img):
    # Ensure the last dimension represents channels (RGB)
    img = img.permute(2, 0, 1)  # Move channel dimension to the correct position

    # Resize image to 299x299 as required by Inception v3
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
    ])
    img = preprocess(img)

    # Manually normalize image as expected by Inception v3
    img[0] = (img[0] - 0.485) / 0.229  # Normalize red channel
    img[1] = (img[1] - 0.456) / 0.224  # Normalize green channel
    img[2] = (img[2] - 0.406) / 0.225  # Normalize blue channel

    return img

def calculate_activation_statistics(images, model):
    model.eval()
    activations = model(images)[0]
    mu = torch.mean(activations, dim=0)
    sigma = torch_cov(activations, rowvar=False)
    return mu, sigma

def torch_cov(m, rowvar=False):
    # Covariance matrix calculation
    if m.dim() > 2:
        raise ValueError('m has more than 2 dimensions')
    if m.dim() < 2:
        m = m.view(1, -1)
    if not rowvar and m.size(0) != 1:
        m = m.t()
    fact = 1.0 / (m.size(1) - 1)
    m -= torch.mean(m, dim=1, keepdim=True)
    mt = m.t()  # if complex: mt = m.t().conj()
    return fact * m.matmul(mt).squeeze()

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2):
    # Fréchet distance calculation
    diff = mu1 - mu2
    covmean, _ = torch.sqrt(sigma1 * sigma2).sqrt().log().abs().mean(), torch.sqrt(sigma1 * sigma2).sqrt().log().abs().mean()
    return torch.sum(diff * diff + sigma1 + sigma2 - 2 * covmean)

def calculate_fid_score(real_images, fake_images, device='cuda' if torch.cuda.is_available() else 'cpu'):
    # Load Inception v3 model pre-trained on ImageNet
    inception_model = inception_v3(pretrained=True, transform_input=False).to(device)
    inception_model.eval()

    # Preprocess real and fake images
    real_porcessed_imgs = torch.stack([preprocess_image(img) for img in real_images]).to(device)
    fake_processed_imgs = torch.stack([preprocess_image(img) for img in fake_images]).to(device)

    # Compute activations and statistics
    mu_real, sigma_real = calculate_activation_statistics(real_porcessed_imgs, inception_model)
    mu_fake, sigma_fake = calculate_activation_statistics(fake_processed_imgs, inception_model)

    # Compute FID score
    fid_score = calculate_frechet_distance(mu_real, sigma_real, mu_fake, sigma_fake)

    return fid_score.item()

# Example usage:
# Assuming real_img and fake_img are your tensors
# real_img and fake_img should be torch tensors with shapes [300, 320, 320, 3]
fid_score = calculate_fid_score(real_imgs_tensor, fake_imgs_tensor)
print("FID Score:", fid_score)


/home/yair7799/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/yair7799/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.08 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 10.11 GiB is allocated by PyTorch, and 421.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)